In [ ]:
import requests 
import os 
import shutil 
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

## 历届MVP数据

### 可修改字段：需要添加的年份

In [ ]:
years = ["1990","1991","2021","2022"]

In [ ]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"
for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    with open("mvp/{}.html".format(year),"w+",encoding='utf-8') as f:
        f.write(data.text)
    time.sleep(1)

SSLError: HTTPSConnectionPool(host='www.basketball-reference.com', port=443): Max retries exceeded with url: /awards/awards_1989.html (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

In [ ]:
dfs = []
for year in years:
    page = open("mvp/{}.html".format(year),encoding='utf-8')
    soup = BeautifulSoup(page,'lxml')
    # soup.find('tr', class_="over_header")
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(str(mvp_table))[0]
    mvp_df["Year"] = year
    dfs.append(mvp_df)
    page.close()

In [ ]:
mvps = pd.concat(dfs)

In [ ]:
mvps.head(20)

In [ ]:
previous_mvp = pd.read_csv("mvps.csv",header=1,index_col="Unnamed: 0")
previous_mvp.head()

In [ ]:
previous_mvp.columns = mvps.columns
previous_mvp.head()

In [ ]:
mvp_union = pd.concat([previous_mvp,mvps],axis=0)
mvp_union.head()

In [ ]:
print(len(mvps),len(previous_mvp),len(mvp_union))

### 更新mvp数据

In [ ]:
mvp_union.to_csv("mvps.csv")

## 所有球员数据

### 可修改字段：webdriver 地址

In [ ]:
driver = webdriver.Chrome(executable_path='C:/Program Files/Google/Chrome/Application/chromedriver.exe')

In [ ]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [ ]:
for year in years:
    url = player_stats_url.format(year)
    
    data = requests.get(url)
    
    with open("player/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)
    
    time.sleep(1)

In [ ]:
for year in years:
    url = player_stats_url.format(year)
    
    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(1)
    
    with open("player/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(driver.page_source)

In [ ]:
dfs2 = []
for year in years:
    page = open("player/{}.html".format(year),encoding='utf-8')
    soup = BeautifulSoup(page,'lxml')
    # soup.find('tr', class_="over_header")
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs2.append(player_df)
    page.close()

In [ ]:
players = pd.concat(dfs2)
players.head()

In [ ]:
previous_players = pd.read_csv("players.csv",index_col="Unnamed: 0")
previous_players.head()

In [ ]:
players_union = pd.concat([previous_players,players],axis=0)
players_union.head()

In [ ]:
print(len(players),len(previous_players),len(players_union))

### 更新palyers数据

In [ ]:
players_union.to_csv("players.csv")

## 球队数据

In [ ]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [ ]:
for year in years:
    url = team_stats_url.format(year)
    
    data = requests.get(url)
    
    with open("team/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)
        
    time.sleep(1)

In [ ]:
dfs = []
for year in years:
    with open("team/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    # soup.find('tr', class_="thead").decompose()
    e_table = soup.find_all(id="divs_standings_E")[0]
    e_df = pd.read_html(str(e_table))[0]
    e_df["Year"] = year
    e_df["Team"] = e_df["Eastern Conference"]
    del e_df["Eastern Conference"]
    dfs.append(e_df)
    
    w_table = soup.find_all(id="divs_standings_W")[0]
    w_df = pd.read_html(str(w_table))[0]
    w_df["Year"] = year
    w_df["Team"] = w_df["Western Conference"]
    del w_df["Western Conference"]
    dfs.append(w_df)

In [ ]:
teams = pd.concat(dfs)
teams.head()

In [ ]:
previous_teams = pd.read_csv("teams.csv",index_col="Unnamed: 0")
previous_teams.head()

In [ ]:
teams_union = pd.concat([previous_teams,teams],axis=0)
teams_union.head()

In [ ]:
print(len(teams),len(previous_teams),len(teams_union))

### 更新teams数据

In [ ]:
teams_union.to_csv("teams.csv")